In [ ]:
def make_rk_system(ode_expr, y_func, t_sym, order):
    """
    Converts an nth-order ODE into a system for solve_ivp (RK45-ready).
    
    Parameters:
    - ode_expr: sympy expression for y^{(n)} explicitly (e.g. y'' = RHS)
    - y_func: sympy.Function('y')(t), the dependent variable
    - t_sym: independent variable (usually t)
    - order: order of the ODE (e.g., 2 for y'')
    
    Returns:
    - A callable function f(t, y) for solve_ivp
    """
    # Symbolic variables u0 = y, u1 = y', ..., u_{n-1} = y^{(n-1)}
    u = sp.symbols(f'u0:{order}')  # Creates (u0, u1, ..., u_{n-1})

    # Substitution map: y(t), y'(t), ... => u0, u1, ...
    subs = {sp.diff(y_func, t_sym, i): u[i] for i in range(order)}
    subs[y_func] = u[0]

    # Construct the system: dy/dt = [u1, u2, ..., f(t, u0,...,u_{n-1})]
    rhs_exprs = list(u[1:])  # u1, u2, ..., u_{n-2}
    rhs_exprs.append(ode_expr.subs(subs))  # u_{n-1}' = f(t, u0,...)

    # Convert to numeric function
    rhs_func = lambdify((t_sym, u), rhs_exprs, modules='numpy')

    # Return function suitable for solve_ivp
    def system(t, y):
        return np.array(rhs_func(t, *y), dtype=float)

    return system
